# Bike counter forecasting - Data Stream Consumer 

## Goal
The goal of this notebook is to perform Task 1 and 2 : that is to apply the persistence and average models on the stream produced by the Producer.

This processing will be done separetaly for each models :
- **persistance**
- **average n=2**
- **average n=3**
- **average n=4**

Also for a some model, say average n=2, we will maintain 1 state/model per Sensor. This will be enabled by the *updateStateByKey()* function.
Each state will contain :
1. the key : the name of the Sensor
2. the previous values in an array
3. the number of predictions
4. the sum of the squarred error (SSE)
5. the number of time we have the same prediction error

Performance Results can be seen in **Model_Racing_Plots.ipynb**



## Common imports

In [1]:
import pandas as pd
import numpy as np
import ast
from pyspark import Row
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from itertools import combinations
from pyspark.sql.types import StructField, IntegerType, FloatType
import os
import math 
from pyspark.ml.evaluation import RegressionEvaluator

os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 ' + \
                                    '--conf spark.driver.memory=1g  pyspark-shell '
streaming_path = "streaming"

## Spark session construction

In [2]:
#This function creates a connection to a network socket
#You may change the batch interval
#The function returns the Spark context, Spark streaming context, and DStream object
SOCKET = 2222
def getDStream(spark, batch_interval=10):

    # Get Spark context
    sc = spark.sparkContext
    sc.setLogLevel("ERROR")

    #Create streaming context, with required batch interval
    ssc = StreamingContext(sc, batch_interval)

    #Checkpointing needed for stateful transforms
    ssc.checkpoint("checkpoint")
    
    # Create a DStream that represents streaming data from a network socket
    # See https://spark.apache.org/docs/latest/streaming-programming-guide.html#a-quick-example
    dstream = ssc.socketTextStream("localhost", SOCKET)
    
    return [sc,ssc,dstream]

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("bike_forecasting") \
    .getOrCreate()
sc = spark.sparkContext

23/05/15 21:02:15 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.129.3 instead (on interface enp38s0)
23/05/15 21:02:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/15 21:02:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/15 21:02:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Stateful Processing of persistance model

In [4]:
#Retrieve first the different sensors to define the different keys of the states
df = spark.read.option("header", True).csv("data.csv")
different_sensors = df.filter("Date =='2022-01-01'").filter(col("Time Gap") == 1).select(col("Sensor")).distinct().toPandas().values
different_sensors

array([['CB02411'],
       ['CEE016'],
       ['CJM90'],
       ['CEK049'],
       ['CB1101'],
       ['CB2105'],
       ['CB1142'],
       ['CLW239'],
       ['CB1699'],
       ['CB1143'],
       ['CJE181'],
       ['CB1599'],
       ['CEV011'],
       ['CVT387'],
       ['CEK18'],
       ['COM205'],
       ['CEK31'],
       ['CAT17']], dtype=object)

In [5]:
def persistance_prediction(previous_values):
    """
    Perform the persistence model i.e. predict the new value to be the same of the preceeding value.
    @param previous_values : an np.array with size 1 containing the previous value 
    """
    return previous_values[0]
def average_model(previous_values):
    """
    Perform the average model i.e. predict the new value to be the average of the previous values seen.
    @param previous_values : an np.array with size n.
    Note that for each N, the size of previous_values will be different.
    """
    return previous_values.mean()
def set_state_from_x(x, previous_value_size, prediction_function):
    """
    Define the state value (key not included) from x (the row received from producer).
    @param previous_value_size : the number of previous value we want to keep track of. (Equal to the n parameter in the statement)
    @param prediction_function : the prediction function, either persistance_prediction or average_model
    """
    previous_values = np.zeros(previous_value_size)
    nb_predictions = 0
    SSE = 0
    last_prediction = 0
    nb_same_SSE = 0
    return (x[0],last_prediction, previous_values,nb_predictions,SSE, prediction_function, nb_same_SSE)
def updateFunction(newValues, state):
    """
    Update the state of the corresponding sensor with newValues.
    Perform the prediction and updtate the metrics
    @param newValues : the new values from the stream : one row of the corresponding sensor
    @param state : the current state to be updated
    """
    nb_values = len(newValues)
    if nb_values > 0:
        for line in newValues:            
            key, last_prediction, previous_values, nb_predictions, SSE, prediction_function, nb_same_SEE = state 
            true_value = line[1][4]
            #use the prediction function defined by the state
            prediction = prediction_function(previous_values) 
            SSE += math.pow(float(true_value)-float(prediction), 2)
            nb_predictions +=1
            #replace the oldest value seen by the true value, works thanks to modulo
            previous_values[nb_predictions%len(previous_values)] = true_value
            date = line[1][1]
           
            state = (key, prediction, previous_values, nb_predictions, SSE, prediction_function,0)
    else:
        #this block is used to set the state to none if we have 5 time the same SEE (we do not receive new data 5 times in a row)
        key, last_prediction, previous_values, nb_predictions, SSE, prediction_function, nb_same_SEE = state 
        nb_same_SEE +=1
        if nb_same_SEE >= 5:
            return None
        state = (key, last_prediction, previous_values, nb_predictions, SSE, prediction_function, nb_same_SEE)
    return state

## Persistance model computation

In [6]:
batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_state_from_x(x, 1, persistance_prediction)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)


dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunction, initialRDD=initialStateRDD)

def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the is stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/Task1/trash")


updatedState.foreachRDD(lambda x : save_to_file(x))


In [7]:
ssc.start()
ssc.awaitTermination(10)

-------------------------------------------
Time: 2023-05-15 21:02:36
-------------------------------------------



-------------------------------------------
Time: 2023-05-15 21:02:38
-------------------------------------------
('CAT17', ('CAT17', ['2120256', '2022-04-16', '1', 'CAT17', '1.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB02411', ('CB02411', ['2120257', '2022-04-16', '1', 'CB02411', '3.0', '22.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0'

-------------------------------------------
Time: 2023-05-15 21:02:40
-------------------------------------------



-------------------------------------------
Time: 2023-05-15 21:02:42
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

In [ ]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/15 21:02:46 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-15 21:02:44
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

## Average Model

### N = 2

In [14]:
batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_state_from_x(x, 2, average_model)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)
dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunction, initialRDD=initialStateRDD)
def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/Task1/avg_2")


updatedState.foreachRDD(lambda x : save_to_file(x))

In [15]:
ssc.start()

-------------------------------------------
Time: 2023-05-14 13:51:00
-------------------------------------------
('CB02411', ('CB02411', ['2120257', '2022-04-16', '1', 'CB02411', '3.0', '22.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB2105', ('CB2105', ['2120263', '2022-04-16', '1', 'CB2105', '7.0', '19

-------------------------------------------
Time: 2023-05-14 13:51:02
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:51:04
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

-------------------------------------------
Time: 2023-05-14 13:51:06
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 13:51:08
-------------------------------------------
('CAT17', ('CAT17', ['2147904', '2022-05-02', '1', 'CAT17', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB02411', ('CB02411', ['2147905', '2022-05-02', '1', 'CB02411', '49.0', '17.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1101', ('CB1101', ['2147906', '2022-05-02', '1', 'CB1101', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1142', ('CB1142', ['2147907', '2022-05-02', '1', 'CB1142', '5.0', '20.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1143', ('CB1143', ['2147908', '2022-05-02', '1', 'CB1143', '5.0', '23.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1599', ('CB1599', ['2147909', '2022-05-02', '1', 'CB1599', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1699', ('CB1699', ['2147910', '2022-05-02', '1', 'CB1699', '0.0', '-1.0

...

-------------------------------------------
Time: 2023-05-14 13:53:24
-------------------------------------------
('CB2105', ('CB2105', ['2570371', '2023-01-01', '47', 'CB2105', '6.0', '25.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CEE016', ('CEE016', ['2570372', '2023-01-01', '47', 'CEE016', '8.0', '17.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CEK049', ('CEK049', ['2570373', '2023-01-01', '47', 'CEK049', '7.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CEK18', ('CEK18', ['2570374', '2023-01-01', '47', 'CEK18', '4.0', '15.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CEK31', ('CEK31', ['2570375', '2023-01-01', '47', 'CEK31', '5.0', '20.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CEV011', ('CEV011', ['2570376', '2023-01-01', '47', 'CEV011', '2.0', '21.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CJE181', ('CJE181', ['2570377', '2023-01-01', '47', 'CJE181', '2.0', '17.0', '1', '1', '2023', '52

-------------------------------------------
Time: 2023-05-14 13:53:26
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:53:28
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:53:30
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:53:32
-------------------------------------------



In [16]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/14 13:54:27 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-14 13:53:34
-------------------------------------------



### N = 3

In [17]:
batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_state_from_x(x, 3, average_model)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)
dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunction, initialRDD=initialStateRDD)
def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/Task1/avg_3")


updatedState.foreachRDD(lambda x : save_to_file(x))

Then, we start the data consumption. At each `batch_interval`, the data sent by the producer is collected.

In [18]:
ssc.start()


-------------------------------------------
Time: 2023-05-14 13:54:46
-------------------------------------------
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB2105', ('CB2105', ['2120263', '2022-04-16', '1', 'CB2105', '7.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CEE016', ('CEE016', ['2120264', '2022-04-16', '1', 'CEE016', '1.0', '19.0'

-------------------------------------------
Time: 2023-05-14 13:54:48
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:54:50
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

-------------------------------------------
Time: 2023-05-14 13:54:52
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 13:54:54
-------------------------------------------
('CAT17', ('CAT17', ['2147904', '2022-05-02', '1', 'CAT17', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB02411', ('CB02411', ['2147905', '2022-05-02', '1', 'CB02411', '49.0', '17.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1101', ('CB1101', ['2147906', '2022-05-02', '1', 'CB1101', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1142', ('CB1142', ['2147907', '2022-05-02', '1', 'CB1142', '5.0', '20.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1143', ('CB1143', ['2147908', '2022-05-02', '1', 'CB1143', '5.0', '23.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1599', ('CB1599', ['2147909', '2022-05-02', '1', 'CB1599', '0.0', '-1.0', '122', '2', '2022', '18', '2022-122', '18', '2022-18']))
('CB1699', ('CB1699', ['2147910', '2022-05-02', '1', 'CB1699', '0.0', '-1.0

...

-------------------------------------------
Time: 2023-05-14 13:57:06
-------------------------------------------
('CAT17', ('CAT17', ['2716416', '2023-03-27', '1', 'CAT17', '0.0', '-1.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB02411', ('CB02411', ['2716417', '2023-03-27', '1', 'CB02411', '1.0', '19.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1101', ('CB1101', ['2716418', '2023-03-27', '1', 'CB1101', '3.0', '13.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1142', ('CB1142', ['2716419', '2023-03-27', '1', 'CB1142', '1.0', '22.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1143', ('CB1143', ['2716420', '2023-03-27', '1', 'CB1143', '2.0', '17.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1599', ('CB1599', ['2716421', '2023-03-27', '1', 'CB1599', '0.0', '-1.0', '86', '2', '2023', '13', '2023-86', '13', '2023-13']))
('CB1699', ('CB1699', ['2716422', '2023-03-27', '1', 'CB1699', '0.0', '-1.0', '86', '2',

-------------------------------------------
Time: 2023-05-14 13:57:08
-------------------------------------------
('CAT17', ('CAT17', ['2569536', '2023-01-01', '1', 'CAT17', '1.0', '9.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB02411', ('CB02411', ['2569537', '2023-01-01', '1', 'CB02411', '8.0', '17.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1101', ('CB1101', ['2569538', '2023-01-01', '1', 'CB1101', '17.0', '15.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1142', ('CB1142', ['2569539', '2023-01-01', '1', 'CB1142', '12.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1143', ('CB1143', ['2569540', '2023-01-01', '1', 'CB1143', '16.0', '18.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1599', ('CB1599', ['2569541', '2023-01-01', '1', 'CB1599', '3.0', '19.0', '1', '1', '2023', '52', '2023-1', '52', '2023-52']))
('CB1699', ('CB1699', ['2569542', '2023-01-01', '1', 'CB1699', '2.0', '16.0', '1', '1', '2023', '5

-------------------------------------------
Time: 2023-05-14 13:57:10
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:57:12
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:57:14
-------------------------------------------



To stop the data consumption, we stop the data stream object.

In [19]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

23/05/14 13:58:12 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

-------------------------------------------
Time: 2023-05-14 13:57:16
-------------------------------------------



### N = 4

In [20]:
batch_interval = 2
[sc,ssc,dstream] = getDStream(spark, batch_interval)
initialStateRDD = sc.parallelize([(x[0], set_state_from_x(x, 4, average_model)) for x in different_sensors]) 
dataS=dstream.map(ast.literal_eval)
dataS=dataS.flatMap(lambda x: [(x[3],(x[3],x))]) #x[3] contains the key (sensor)
dataS.pprint()
updatedState = dataS.updateStateByKey(updateFunction, initialRDD=initialStateRDD)
def save_to_file(rdd):
    """
    Update the state on a file to retrieve data after the stream completed
    """
    # coalesce the RDD into a single partition
    merged_rdd = rdd.coalesce(1)
    # save the merged RDD to a text file
    if not merged_rdd.isEmpty():
        merged_rdd.map(lambda x: f"{x[0]},{(x[1][4]/(x[1][3] if x[1][3] != 0 else 1)):.2f},{x[1][4]},{x[1][3]}") \
            .saveAsTextFile("streaming/Task1/avg_4")

updatedState.foreachRDD(lambda x : save_to_file(x))

In [21]:
ssc.start()

-------------------------------------------
Time: 2023-05-14 13:58:52
-------------------------------------------
('CB1101', ('CB1101', ['2120258', '2022-04-16', '1', 'CB1101', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1142', ('CB1142', ['2120259', '2022-04-16', '1', 'CB1142', '3.0', '21.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1143', ('CB1143', ['2120260', '2022-04-16', '1', 'CB1143', '1.0', '27.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1599', ('CB1599', ['2120261', '2022-04-16', '1', 'CB1599', '1.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB1699', ('CB1699', ['2120262', '2022-04-16', '1', 'CB1699', '0.0', '-1.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CB2105', ('CB2105', ['2120263', '2022-04-16', '1', 'CB2105', '7.0', '19.0', '106', '7', '2022', '15', '2022-106', '15', '2022-15']))
('CEE016', ('CEE016', ['2120264', '2022-04-16', '1', 'CEE016', '1.0', '19.0'

-------------------------------------------
Time: 2023-05-14 13:58:54
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 13:58:56
-------------------------------------------
('CAT17', ('CAT17', ['2123712', '2022-04-18', '1', 'CAT17', '2.0', '17.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB02411', ('CB02411', ['2123713', '2022-04-18', '1', 'CB02411', '2.0', '28.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1101', ('CB1101', ['2123714', '2022-04-18', '1', 'CB1101', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1142', ('CB1142', ['2123715', '2022-04-18', '1', 'CB1142', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1143', ('CB1143', ['2123716', '2022-04-18', '1', 'CB1143', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1599', ('CB1599', ['2123717', '2022-04-18', '1', 'CB1599', '0.0', '-1.0', '108', '2', '2022', '16', '2022-108', '16', '2022-16']))
('CB1699', ('CB1699', ['2123718', '2022-04-18', '1', 'CB1699', '1.0', '17.0'

-------------------------------------------
Time: 2023-05-14 13:58:58
-------------------------------------------
('CAT17', ('CAT17', ['2135808', '2022-04-25', '1', 'CAT17', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB02411', ('CB02411', ['2135809', '2022-04-25', '1', 'CB02411', '3.0', '22.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1101', ('CB1101', ['2135810', '2022-04-25', '1', 'CB1101', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1142', ('CB1142', ['2135811', '2022-04-25', '1', 'CB1142', '2.0', '17.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1143', ('CB1143', ['2135812', '2022-04-25', '1', 'CB1143', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1599', ('CB1599', ['2135813', '2022-04-25', '1', 'CB1599', '0.0', '-1.0', '115', '2', '2022', '17', '2022-115', '17', '2022-17']))
('CB1699', ('CB1699', ['2135814', '2022-04-25', '1', 'CB1699', '1.0', '20.0'

-------------------------------------------
Time: 2023-05-14 13:59:00
-------------------------------------------



...

-------------------------------------------
Time: 2023-05-14 14:01:24
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 14:01:26
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 14:01:28
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 14:01:30
-------------------------------------------



-------------------------------------------
Time: 2023-05-14 14:01:32
-------------------------------------------



In [22]:
ssc.stop(stopSparkContext=False, stopGraceFully=False)

-------------------------------------------
Time: 2023-05-14 14:01:34
-------------------------------------------

23/05/14 14:02:39 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja